# Tarea 2

**Envío del PR inicial: 07.09.2018**

**Aceptación del PR: 14.09.2018**

## 1

Extiendan los operadores `+`, `-`, `*` y `/` para que funcionen cuando una de las dos entradas es un `Dual` y la otra es un `Real` (número cualquiera), sin importar el orden.

In [1]:
"""
    Dual

Definición de los números duales. Los campos internos son
    x   :: Float64      # Valor de la función
    x'  :: Float64      # Valor de su derivada

"""
struct Dual
    x  :: Float64
    x´ :: Float64
end

Dual(c::Real) = Dual(c,0.0)

dual(x0::Real)=Dual(x0,1.0)


#Importamos la definición de los siguientes operadores
import Base:+,-,*,/,^

+(u::Dual,v::Dual)=Dual(u.x+v.x,u.x´+v.x´)

-(u::Dual,v::Dual)=Dual(u.x-v.x, u.x´-v.x´)

*(u::Dual,v::Dual)=Dual( u.x * v.x, u.x´*v.x + u.x*v.x´)

function /(u::Dual , v::Dual)
    y = u.x / v.x
    Dual(y, (u.x´ - y * v.x´)/v.x)
end

function ^(a::Dual , n::Int)
    y = a.x^(n-1)
    Dual(a.x * y , n*y*a.x´)
end

^ (generic function with 63 methods)

In [2]:
#Extenderemos la operación +
+(a::Real, u::Dual) = Dual(a + u.x, u.x´)
+(u::Dual,a::Real) = Dual(a+u.x,u.x´)

#Extenderemos la operación -
-(a::Real,u::Dual) = Dual(a - u.x, -u.x´)
-(u::Dual,a::Real) = Dual(u.x - a, u.x´)

#Extenderemos la operación *
*(a::Real, u::Dual) = Dual(a*u.x , a*u.x´)
*(u::Dual, a::Real) = Dual(a*u.x , a*u.x´)

#Extenderemos la operación /
function /(a::Real,u::Dual)
    Dual(a/u.x , -a*u.x´/u.x^2)
end

/(u::Dual, a::Real) = u*a^(-1)

/ (generic function with 107 methods)

## 2

Construyan 2 tests para cada una de las operaciones aritméticas definidas para los `Dual`es, incluyendo las que definieron en el ejercicio 1.

In [4]:
import Test.@test   

#Funciona para cuando tnemos más de dos elementos: u + v + w = (u + v) + w

In [6]:
#Tests para operación +
@test Dual(1,1) + pi + Dual(-1,-1) == Dual(pi,0)
@test 1 + Dual(2,2) + Dual(1,1) == Dual(4,3) 

Test Passed

In [5]:
#Tests para operación -
@test Dual(1,1) - pi - Dual(-1,-1) == Dual(2-pi,2)
@test 1 - Dual(2,2) - Dual(1,1) == Dual(-2,-3) 

Test Passed

In [7]:
#Tests para operación *
@test Dual(1,3)*5*Dual(-1,1) == Dual(-5,-10) && 5*Dual(1,3)*Dual(-1,1) == Dual(-5,-10)

Test Passed

In [8]:
#Tests para operación /
@test Dual(2,1)/0.5/Dual(3,4) == Dual(4/3,-10/9)

@test 4/Dual(2,1)/Dual(3,4) == Dual(2/3,-11/9)

Test Failed at In[8]:2
  Expression: (Dual(2, 1) / 0.5) / Dual(3, 4) == Dual(4 / 3, -10 / 9)
   Evaluated: Dual(1.3333333333333333, -1.111111111111111) == Dual(1.3333333333333333, -1.1111111111111112)


Test.FallbackTestSetException: There was an error during testing

In [9]:
#Tests para la operación ^
@test Dual(2.5,3)^3 == Dual(2.5^3,9*6.25) 
@test Dual(25,16)^1 == Dual(25, 16) 

Test Passed

In [10]:
#Test passed, pero hay un error durante el testing, es mejor en líneas separadas
@test Dual(2.5,3)^3 == Dual(2.5^3,9*6.25) && @test Dual(25,16)^1 == Dual(25,16) 

Error During Test at In[10]:2
  Expression evaluated to non-Boolean
  Expression: Dual(2.5, 3) ^ 3 == Dual(2.5 ^ 3, 9 * 6.25) && #= In[10]:2 =# @test(Dual(25, 16) ^ 1 == Dual(25, 16))
       Value: Test Passed


Test.FallbackTestSetException: There was an error during testing

## 3

Sobrecarguen la función `show` (que está en Base) para que cuando se impriman los duales aparezcan con la notación $\varepsilon$ que usamos al principio de este notebook, por ejemplo, el resultado de `Dual(1,2)` debe ser parecido a `1.0 + 2.0 ε`.

## 4

Obtengan la derivada de 
$$g(x) = \frac{3x^2-8x+5}{7x^3-1}$$
en $x_0=1$.

## 5

- Recordando la regla de la cadena(!!!), extiendan el usar los `Dual` a las funciones `sqrt`, `exp`, `log`, `sin`, `cos`, `sinh` y `cosh`. 

- Muestren que las cosas dan los resultados que esperan usando pruebas como hicieron en el ejercicio 2.

- Calculen la derivada de $h(x) = \sin\Big(x^3 - \frac{2}{x^6}\Big)$ en $x_0 = 2$. ¿Qué tan preciso es el resultado?(Pueden usar cualquier otra manera de obtener el resultado correcto, sólo tienen que ser claros en la explicación.)

- Dibujen, para $x_0 \in [1,5]$ la función $h'(x)$.

## 6

Implementen el método de Newton para una función arbitraria $f(x)$ en una variable, explotando el uso de los `Dual`. 

- Obtengan, usando su función para el método de Newton, las raices del polinomio de Wilkinson 

$$
W_{6}(x) = (x-1)(x-2)(x-3)(x-4)(x-5)(x-6),
$$ 

usando como iterados iniciales $x_0=2.2$ y $x_0=2.45$.
    
- Hagan lo mismo que en el inciso anterior para $g(x) = x \sin(2x)$, con $x_0=0.7$.

## 7

Argumenten qué podrían hacer para extender la idea de los `Dual` y calcular derivadas aún más altas. Como caso concreto, piensen en querer obtener la derivada 18 de funciones como las que hemos usado arriba